In [3]:
import lxml.html
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import re
import string
import warnings
import os
import hashlib
warnings.filterwarnings('ignore')

In [41]:
def get_recipe_keys(page_numbers):
    recipe_keys = []
    for page_number in page_numbers:
        url = 'http://www.epicurious.com/search?content=recipe&page=%d&sort=highestRated' % page_number
        r = requests.get(url)
        all_tags = bs(r.content)
        recipies = np.array([x.get_text().strip() for x in all_tags.find_all("a")])[np.arange(1,105,6)]
        recipe_key = ["-".join(x.lower().split(" ")) for x in recipies]
        recipe_keys.append(recipe_key)
    return([item for sublist in recipe_keys for item in sublist])

In [37]:
def get_recipe_reviews(recipe):
    url = 'http://www.epicurious.com/recipes/food/reviews/%s' % recipe
    r = requests.get(url)
    all_tags = bs(r.content)
    tot_reviews = int(all_tags.find(class_ = "reviews-count").get_text())
    tot_review_pages = len(range(0,tot_reviews,10))
    review_page = 1
    review_list = []
    while(review_page <= tot_review_pages):
        #print(review_page, end="|")
        url = 'http://www.epicurious.com/recipes/food/reviews/%s?page=%s' % (recipe, review_page)
        r = requests.get(url)
        all_tags = bs(r.content)
        review_list.append([x.find("p").get_text().strip() for x in all_tags.find_all("div" ,class_ = "review-text")])
        review_page += 1
    return([item for sublist in review_list for item in sublist])


In [42]:
def get_recipe_stats(recipe_name):
    recipe_dict = {}
    recipe_dict['recipe_id'] = hash(recipe_name) % (10 ** 7)
    url = 'http://www.epicurious.com/recipes/food/views/%s' % recipe_name
    try:
        r = requests.get(url)
    except:
        return(np.nan)
    all_tags = bs(r.content)
    recipe_dict['name'] = all_tags.find("h1").get_text().strip()
    try:
        recipe_dict['cooking_stats'] = [int(re.findall(string = x.get_text(), pattern = "^\d+")[0]) for x in all_tags.find_all("dd")]
        servings = cooking_stats[0]
        active_time = cooking_stats[1]
        total_time = cooking_stats[2]
    except:
        pass
    
    nutritional_labels = [x.get_text().strip() for x in all_tags.find_all(class_ = "nutri-label")]
    nutritional_data = [x.get_text().strip() for x in all_tags.find_all(class_ = "nutri-data")]
    recipe_dict['nutrition'] = dict(zip(nutritional_labels, nutritional_data))
    #For page reviews
    try:
        recipe_dict['reviews'] = get_recipe_reviews(recipe_name)
    except:
        pass
    
    return(recipe_dict)

In [43]:
recipe_names = get_recipe_keys(range(1,100))

In [39]:
recipes = [get_recipe_stats(x) for x in recipe_names]

pasta-with-shrimp-in-tomato-cream|green-goddess-buddha-bowl|crispy-chicken-thighs-with-spring-vegetables|one-skillet-steak-and-spring-veg-with-spicy-mustard|one-skillet-roasted-butternut-squash-with-spiced-chickpeas|pasta-with-ramp-pesto-and-guanciale|warm-chicken-salad-with-creamy-dill-dressing|spiced-moroccan-vegetable-soup-with-chickpeas,-cilantro,-and-lemon|granola-bark|mini-beef-and-mushroom-patties|spring-risotto|classic-cream-cheese-frosting|soy-sauce-and-citrus-marinated-chicken|turkish-lamb-chops-with-sumac,-tahini,-and-dill|gluten-free-rice-buns|mixed-citrus-daiquiri|gingery-chicken-soup-with-zucchini-“noodles”|cinnamon-white-hot-chocolate|

In [40]:
recipes[2]

{'cooking_stats': [4, 45, 45],
 'name': 'Crispy Chicken Thighs with Spring Vegetables',
 'nutrition': {'Calories': '1006',
  'Carbohydrates': '29 g(10%)',
  'Cholesterol': '344 mg(115%)',
  'Fat': '69 g(107%)',
  'Fiber': '6 g(24%)',
  'Monounsaturated Fat': '31 g',
  'Polyunsaturated Fat': '14 g',
  'Protein': '62 g(125%)',
  'Saturated Fat': '18 g(90%)',
  'Sodium': '1373 mg(57%)'},
 'recipe_id': 348161,
 'reviews': ['This was absolutely delicious! I made this for the two boys I nanny for, and was so glad I made all the vegetables, because none of the adults could stop eating them. The chicken came out perfectly and the sauce was incredible. This goes in the trove of treasured guaranteed-hit recipes. Thank you!',
  'This is great, very tasty. Is it really 1000 calories per serving? That seems a bit high.']}